# Альтернативные способы предсказаний

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from implicit.nearest_neighbours import ItemItemRecommender 

In [2]:
# импорт модулей src.utils и src.recommenders

from src.utils import precision_at_k, prefilter_items, postfilter_items
from src.utils import get_similar_items_recommendation, popular_items, gen_dicts

### Кейс 1
Берем топ-5 популярных покупок юзера --> находим по 1 наиболее близкому товару для них --> рекомендуем

In [3]:
# читаем данные из файла
data = pd.read_csv('transaction_data.csv')

# приводим названия столбцов к нижнему регистру и переименовываем два столбца
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3
# делим выборку на train и test
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# в тестовых данных для каждого пользователя находим список купленных товаров
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

# в обучающей выборке берем только 5000 самых популярных товаров
data_train = popular_items(data=data_train)

In [4]:
# строим матрицу c_ui
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
c_ui = user_item_matrix 
user_item_matrix = bm25_weight(user_item_matrix.T).T  # применям к c_ui матрице bm25 веса 

In [5]:
%%time

# обучаем ALS модель на матрице c_ui
model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                              show_progress=True)


Wall time: 1.17 s


In [6]:
# делаем предсказание с помощью построенной модели для каждого пользователя
res = get_similar_items_recommendation(model, data_train, c_ui, N=5)

In [7]:
res.head(10)

,user_id,similar_recommendation
0,1,"[940631, 949620, 1132231, 986021, 15926844]"
1,2,"[836793, 960318, 1133018, 880888, 819978]"
2,3,"[892008, 960318, 949965, 1076875, 901543]"
3,4,"[1074754, 1084310, 945998, 932863, 930344]"
4,5,"[942778, 946839, 833025, 12262778, 1105301]"
5,6,"[948650, 5569845, 889731, 934639, 1037863]"
6,7,"[922307, 938700, 863030, 1015247, 9526411]"
7,8,"[1110843, 5569845, 1133018, 904360, 889731]"
8,9,"[881708, 1098066, 982493, 1049133, 1008032]"
9,10,"[957736, 1128647, 904375, 945779, 1137688]"


### Кейс 2. Вашим друзьям нравится / Похожим пользователям нравится / ...

Пользователь --> похожие пользователи --> рекомендовать те товары, которые купили похожие пользователи

In [8]:
userids = c_ui.index.values
itemids = c_ui.columns.values

id_to_userid, userid_to_id = gen_dicts(userids)
id_to_itemid, itemid_to_id = gen_dicts(itemids)

In [9]:
model.similar_users(userid_to_id[10], N=6)

[(9, 1.0),
 (1549, 0.78354),
 (790, 0.73201895),
 (2237, 0.7301203),
 (612, 0.72247094),
 (2222, 0.71856236)]

In [10]:
%%time

own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

own.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = own.recommend(userid=354, 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

recs


Wall time: 446 ms


[(1181, 198554.18982189914),
 (938, 181154.4700471918),
 (4997, 170564.3245009548),
 (2628, 162971.69347370384),
 (2410, 148948.40093582234)]

In [11]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=True)]
    return res


get_recommendations(user=10, model=own, N=1)

[1128900]

In [12]:
model.recommend(userid=userid_to_id[354], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=True)

[(1468, 1.251161877143295),
 (3607, 1.2355227262517756),
 (2747, 1.2247883902714203),
 (3487, 1.2100478229616412),
 (4505, 1.1923643880165884)]